# 📊 Notebook de Análisis y Persistencia de `sales_data_sample`

Este notebook en **Databricks + PySpark** carga, procesa y analiza el archivo 
`sales_data_sample.csv`.  Información del dataset: https://www.kaggle.com/datasets/kyanyoga/sample-sales-data

Incluye:
- ✅ Lectura 
- ✅ Creación de vistas temporales y persistentes  
- ✅ Consultas con joins y agregaciones  
- ✅ Ejemplos de funciones de ventana explicadas en detalle  

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Ruta por defecto (ajusta si tu CSV está en otra ubicación)
file_path = "/Volumes/workspace/dbtest/dataclase3"

# Lectura del CSV con header=True y separador ','
df_sales = spark.read.option("header", "true") \
.option("sep", ",") \
.option("inferSchema", "true") \
.csv(file_path)

# Mostrar esquema y primeras filas
print("Esquema detectado:")
df_sales.printSchema()

# En Databricks es práctico usar display()
display(df_sales.limit(10))

## Creación de Vistas Temporales y Globales
En Spark/Databricks podemos exponer DataFrames como vistas SQL:

- **Vista Temporal**: solo disponible en la sesión actual.  
- **Vista Global Temporal**: accesible desde todas las sesiones dentro de la aplicación Spark.  

In [0]:

# Vista temporal
df_sales.createOrReplaceTempView("vw_sales_temp")

# Vista global
df_sales.createOrReplaceGlobalTempView("vw_sales_global")

print("✅ Vistas creadas: vw_sales_temp (temporal) y global_temp.vw_sales_global (global)")

## Creación de Tabla Persistente en Catálogo
Creamos un esquema llamado `sales_analytics` y guardamos la tabla en formato **Delta**.  
Este formato soporta **ACID transactions, versioning y optimizaciones**.

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS sales_analytics")

df_sales.write.format("delta").mode("overwrite").saveAsTable("sales_analytics.sales_raw")

print("✅ Tabla persistente creada: sales_analytics.sales_raw")

In [0]:
%sql
-- SELECT * 
-- FROM sales_analytics.sales_raw;

SELECT
    PRODUCTLINE,
    YEAR_ID,
    QTR_ID,
    SUM(SALES) AS total_sales,
    COUNT(DISTINCT ORDERNUMBER) AS total_orders,
    ROUND(AVG(PRICEEACH), 2) AS avg_unit_price,
    ROUND(SUM(SALES) / COUNT(DISTINCT ORDERNUMBER), 2) AS avg_sales_per_order
FROM sales_Analytics.sales_raw
WHERE STATUS = 'Shipped'   -- Filtramos solo pedidos efectivamente enviados
GROUP BY PRODUCTLINE, YEAR_ID, QTR_ID
ORDER BY total_sales DESC
LIMIT 20;

### Agregaciones y Joins
1. Resumen de ventas por cliente  
2. Catálogo de productos únicos  
3. Join con los **Top 10 clientes** y pedidos con `QUANTITYORDERED` superior al promedio global

In [0]:
agg_cliente = df_sales.groupBy("CUSTOMERNAME") \
                     .agg(F.sum("SALES").alias("total_sales"), \
                          F.countDistinct("ORDERNUMBER").alias("order_count")) \
                     .orderBy(F.desc("total_sales"))

display(agg_cliente.limit(10))

# Tabla de Productos únicos
productos = df_sales.select("PRODUCTCODE", "PRODUCTLINE", "MSRP", "PRICEEACH").dropDuplicates(["PRODUCTCODE"])

# Se crea lista de los top 10 clientes
top10_customers = [r["CUSTOMERNAME"] for r in agg_cliente.limit(10).collect()] # Lista en Python
avg_qty = df_sales.agg(F.avg("QUANTITYORDERED")).collect()[0][0]

df_top_orders = df_sales.filter((F.col("CUSTOMERNAME").isin(top10_customers)) & (F.col("QUANTITYORDERED") > avg_qty))

display(df_top_orders.limit(20))

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Datos simulados: ventas diarias por vendedor
data = [
    ("Ana", "2025-09-01", 200),
    ("Ana", "2025-09-02", 350),
    ("Ana", "2025-09-03", 400),
    ("Ana", "2025-09-04", 150),
    ("Luis", "2025-09-01", 500),
    ("Luis", "2025-09-02", 300),
    ("Luis", "2025-09-03", 450),
    ("Luis", "2025-09-04", 250),
    ("Marta", "2025-09-01", 600),
    ("Marta", "2025-09-02", 700),
    ("Marta", "2025-09-03", 500),
]

columns = ["vendedor", "fecha", "ventas"]

df = spark.createDataFrame(data, columns)

print("=== Datos de ejemplo ===")
df.show()

In [0]:
# Definir ventana: partición por vendedor y orden por fecha
window_spec = Window.partitionBy("vendedor").orderBy("fecha")

1. Cálculo con filas anteriores o siguientes (lag, lead)

- Se crea una columna con las ventas del día anterior y otra con el día siguiente para cada vendedor.
- Ejemplo: Ana (2025-09-02) → ventas_dia_anterior = 200, ventas_dia_siguiente = 400.

In [0]:
# ==============================
# 1. Cálculo con filas anteriores o siguientes
# ==============================
df_lag = df.withColumn("ventas_dia_anterior", F.lag("ventas", 1).over(window_spec)) \
           .withColumn("ventas_dia_siguiente", F.lead("ventas", 1).over(window_spec))

print("=== Cálculo con filas anteriores y siguientes ===")
df_lag.show()

2. Cálculos acumulativos (sum con window)

- Se calcula la suma acumulada de ventas por vendedor a lo largo de las fechas.
- Ejemplo: Ana (2025-09-03) → ventas acumuladas = 200 + 350 + 400 = 950.

In [0]:
# ==============================
# 2. Cálculos acumulativos
# ==============================
df_cumulative = df.withColumn("ventas_acumuladas", F.sum("ventas").over(window_spec))

print("=== Cálculo acumulativo de ventas ===")
df_cumulative.show()

3. Funciones de ranking (row_number, rank, dense_rank)

- Se asignan posiciones de orden a las filas de cada vendedor:

  - row_number() → enumeración simple (1,2,3...).
  - rank() → ranking con saltos si hay empates.
  - dense_rank() → ranking sin saltos.

In [0]:
# ==============================
# 3. Funciones de ranking
# ==============================
df_rank = df.withColumn("row_num", F.row_number().over(window_spec)) \
            .withColumn("rank", F.rank().over(window_spec)) \
            .withColumn("dense_rank", F.dense_rank().over(window_spec))

print("=== Ranking de ventas por fecha ===")
df_rank.show()

In [0]:
# ==============================
# Ejemplo de rank() vs dense_rank()
# ==============================

# Datos simulados: puntajes de estudiantes en un examen
data = [
    ("Ana", 95),
    ("Luis", 90),
    ("Marta", 95),
    ("Pedro", 88),
    ("Lucía", 90),
    ("Jorge", 85),
    ("Sofía", 95),
]

columns = ["estudiante", "puntaje"]

df = spark.createDataFrame(data, columns)

print("=== Datos de ejemplo ===")
df.show()

# Definir ventana: ordenamos por puntaje descendente
window_spec = Window.orderBy(F.desc("puntaje"))

# Aplicar rank() y dense_rank()
df_rank = df.withColumn("rank", F.rank().over(window_spec)) \
            .withColumn("dense_rank", F.dense_rank().over(window_spec))

print("=== Comparación entre rank() y dense_rank() ===")
df_rank.show()

## Funciones de Ventana (Window Functions)
Aplicamos **row_number, dense_rank, lag y lead** para obtener diferentes perspectivas:

- `row_number`: la venta más alta por cliente  
- `dense_rank`: ranking de productos por línea de producto  
- `lag/lead`: comparación de ventas con pedido anterior y siguiente  

In [0]:
# row_number
w_customer_sales = Window.partitionBy("CUSTOMERNAME").orderBy(F.desc("SALES"))
df_top_sale_per_customer = df_sales.withColumn("rn", F.row_number().over(w_customer_sales)) \
                                     .filter(F.col("rn") == 1)

display(df_top_sale_per_customer.limit(10))

# dense_rank
prod_agg = df_sales.groupBy("PRODUCTCODE", "PRODUCTLINE") \
                   .agg(F.sum("SALES").alias("total_sale_product"))
prod_agg = prod_agg.withColumn("dense_rank", F.dense_rank().over(Window.partitionBy("PRODUCTLINE").orderBy(F.desc("total_sale_product"))))

display(prod_agg.filter(F.col("dense_rank") <= 5))

# lag / lead
w_customer_date = Window.partitionBy("CUSTOMERNAME").orderBy("ORDERDATE")
df_with_lag_lead = df_sales.withColumn("prev_sales", F.lag("SALES").over(w_customer_date)) \
                          .withColumn("next_sales", F.lead("SALES").over(w_customer_date))

display(df_with_lag_lead.select("CUSTOMERNAME", "ORDERNUMBER", "ORDERDATE", "SALES", "prev_sales", "next_sales").limit(20))